In [110]:
import dimcli
import json
import math
import time
import pandas as pd
import numpy as np

In [2]:
dimcli.login()

DimCli v0.6.1 - Succesfully connected to <https://app.dimensions.ai> (method: dsl.ini file)


#### Search filters

In [138]:
years = [year for year in range(2006,2020)]
years_search = json.dumps(str(years))
FOR_categories = ['11 Medical and Health Sciences']
FOR_categories = json.dumps(FOR_categories)
charite_grid_id = ['grid.6363.0']

In [139]:
def doi_record(grid_ids, years, FOR_categories):
    dsl = dimcli.Dsl()
    data = dsl.query_iterative(f"""search publications 
    where year in {years} and 
    research_orgs in {json.dumps(grid_ids)} and 
    type="article" and 
    journal_lists="PubMed" and
    category_for.name in {FOR_categories}
    return publications[id + doi + pmid + year]""")
    return data.as_dataframe()

#### Get DOIs and dimensions ids for the publications

In [140]:
doi_result = doi_record(charite_grid_id, years, FOR_categories)

1000 / 28669
2000 / 28669
3000 / 28669
4000 / 28669
5000 / 28669
6000 / 28669
7000 / 28669
8000 / 28669
9000 / 28669
10000 / 28669
11000 / 28669
12000 / 28669
13000 / 28669
14000 / 28669
15000 / 28669
16000 / 28669
17000 / 28669
18000 / 28669
19000 / 28669
20000 / 28669
21000 / 28669
22000 / 28669
23000 / 28669
24000 / 28669
25000 / 28669
26000 / 28669
27000 / 28669
28000 / 28669
28669 / 28669


In [143]:
doi_result.head()

,doi,id,pmid,year
0,10.1002/hep.30925,pub.1120832821,31483871,2019
1,10.1080/20008198.2019.1568134,pub.1112116251,30788063,2019
2,10.1111/jch.13787,pub.1123746530,31891443,2019
3,10.1371/journal.pone.0226843,pub.1123756059,31891612,2019
4,10.1080/20008198.2019.1694766,pub.1122945616,31807235,2019


In [144]:
doi_result.to_csv("../../results/Charite_publication_ids_dimensions_2006_19.csv")

#### Check list of publications for citations/references from policy documents

In [145]:
DOI_list = pd.read_csv("../../results/Charite_publication_ids_dimensions_2006_19.csv")

In [146]:
DOI_list.year.value_counts()

2019    2996
2018    2475
2017    2462
2016    2438
2015    2309
2014    2101
2013    2060
2012    2049
2011    1902
2009    1746
2010    1743
2008    1606
2007    1467
2006    1315
Name: year, dtype: int64

In [147]:
def policy_citations(year, doi_results_table):
    
    doi_results_filtered = doi_results_table[doi_results_table.year==year]
    doi_results_filtered = doi_results_filtered.reset_index() #otherwise the batch slicing does not work correctly
    
    policy_results_list = []
    batch_size = 300
    for idx in range(0, len(doi_results_filtered), batch_size):
        ids_batch = doi_results_filtered.loc[idx:idx+batch_size, "id"].tolist()
        dsl = dimcli.Dsl()
        data = dsl.query(f"""search policy_documents 
            where publication_ids in {json.dumps(ids_batch)} 
            return policy_documents[id + publication_ids]
            limit 500""")
        time.sleep(1.5)
        policy_results_list.append(data.as_dataframe())
        
    policy_results_list_city = pd.concat(policy_results_list, ignore_index=True)
    policy_results_list_city["year"] = year 
    return policy_results_list_city
    

In [148]:
policy_citation_results_list = []
for year in years:
    result = policy_citations(year, DOI_list)
    policy_citation_results_list.append(result)

Returned Policy_documents: 9 (total = 9)
Returned Policy_documents: 21 (total = 21)
Returned Policy_documents: 26 (total = 26)
Returned Policy_documents: 12 (total = 12)
Returned Policy_documents: 4 (total = 4)
Returned Policy_documents: 9 (total = 9)
Returned Policy_documents: 20 (total = 20)
Returned Policy_documents: 13 (total = 13)
Returned Policy_documents: 22 (total = 22)
Returned Policy_documents: 12 (total = 12)
Returned Policy_documents: 26 (total = 26)
Returned Policy_documents: 18 (total = 18)
Returned Policy_documents: 16 (total = 16)
Returned Policy_documents: 19 (total = 19)
Returned Policy_documents: 13 (total = 13)
Returned Policy_documents: 0
Returned Policy_documents: 16 (total = 16)
Returned Policy_documents: 27 (total = 27)
Returned Policy_documents: 13 (total = 13)
Returned Policy_documents: 19 (total = 19)
Returned Policy_documents: 9 (total = 9)
Returned Policy_documents: 7 (total = 7)
Returned Policy_documents: 19 (total = 19)
Returned Policy_documents: 19 (tota

In [149]:
policy_citation_results_list = pd.concat(policy_citation_results_list, ignore_index=True)

C:\Users\riedeln\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [150]:
policy_citation_results_list.head()

,id,publication_ids,year
0,policy.656495,"[pub.1029215989, pub.1045508740, pub.103031157...",2006
1,policy.111737,"[pub.1042735104, pub.1054136814, pub.101253037...",2006
2,policy.661632,"[pub.1019348531, pub.1016581362, pub.106923434...",2006
3,policy.4877,"[pub.1000789324, pub.1054142273, pub.102468857...",2006
4,policy.140919,"[pub.1023484094, pub.1046338937, pub.106782023...",2006


In [151]:
policy_citation_results_list.to_csv("../../results/policy_citations.csv")